# Goal

- [ ] review `git` activity from the past `x` days/weeks/months (my main goal is weekly but I want the program to be flexible to use for quarterly and annual review) ('last week done' and 'repo activity' sections)
- [ ] review Apple notes changes?? this is my primary notetaking program but I've tried to access the data before and wasn't able to. It would be great to track changes in pinned notes and flag unchecked tasks without having to enter them ('last week done', 'carryover from last week', capture free text notes that might be worth briefing others about)
- [ ] process a preview weekly for incomplete tasks and prompt user to mark completion as Yes/No/Progress for each task ('last week done' and 'carryover from last week' sections)

I think I already have a way to
- [ ] import this week's meetings from calendar (filter out holiday calendar, events outside of 9am-5pm PT)

# `git` activity review

- probably best to use the [`gitpython`](https://gitpython.readthedocs.io/en/stable/tutorial.html#tutorial-label) module

## setup

In [1]:
# dependencies
from os import listdir, path
from pathlib import Path
from datetime import date, datetime
from dateutil.relativedelta import *
import git
import re

In [2]:
# support methods
def findrepos(gitdir):
    gitdir = path.expanduser(gitdir)
    info = {}
    for dirname in listdir(gitdir):
        dirpath = f"{gitdir}/{dirname}"
        if Path(f"{dirpath}/.git").exists(): info[dirname] = {'path': dirpath}
    assert info
    return info


def checkrepos(info):
    newinfo = info
    for reponame, repoinfo in info.items():
        repo = git.Repo(repoinfo['path'])
        newinfo[reponame]['dirty'] = repo.is_dirty()
        newinfo[reponame]['untracked'] = [
            f for f in repo.untracked_files if 'checkpoint' not in f]
    assert newinfo
    return newinfo


def recentcommits(info, sdate, edate, author=None):
    """Authored datetime is preserved on rebase, and
    we want to include commits from this week that might have been rebasing an earlier commit."""
    newinfo = info
    for reponame, repoinfo in info.items():
        repo = git.Repo(repoinfo['path'])
        if author: newinfo[reponame]['n_other_recent'] = 0
        commits = []
        nother = 0
        for commit in repo.iter_commits():
            tzaware = commit.committed_date + commit.committer_tz_offset
            committed = datetime.fromtimestamp(tzaware)
            if (committed < edate) & (committed >= sdate):
                if author:
                    if author.lower() in commit.author.name.lower(): commits.append(commit)
                    else: newinfo[reponame]['n_other_recent'] += 1
                else: commits.append(commit)
        newinfo[reponame]['recent'] = commits
        if newinfo[reponame]['n_other_recent'] == 0: newinfo[reponame].pop('n_other_recent')
    assert newinfo
    return newinfo


def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))


def formatreponame(reponame, fixedwidth):
    formatted = '+-' + '-' * fixedwidth + '-+\n'
    for line in chunkstring(string=reponame, length=fixedwidth):
        formatted += '| {0:^{1}} |'.format(line, fixedwidth)
    formatted += '\n+-' + '-'*(fixedwidth) + '-+\n'
    return formatted


def formatmessage(msg):
    """I want the first work of the commit message to be title-cased, but not the rest of the message."""
    chunks = msg.strip().split()
    titled = chunks[0].title() + ' ' + ' '.join(chunks[1:])
    if titled[-1] != ".": titled += "."
    return titled


def summarize(reponame, commits):
    if not any(commits): return ""
    summary = ""
    for commit in commits:
        commitdt = datetime.fromtimestamp(commit.committed_date + commit.committer_tz_offset).strftime("%a %d %b")
        nchanges = commit.stats.total
        if commitdt not in summary: summary += f"_Committed: {commitdt}_\n"
        overview = f"* [{commit.hexsha[:8]}]: {formatmessage(msg=commit.message)}"
        overview += f" // Involves {nchanges['files']} file(s), {nchanges['lines']} lines\n"
        summary += overview
    return summary


def summarizerecent(repos):
    fullsummary = ""
    for reponame, repoinfo in repos.items():
        if not ((any(repoinfo['recent'])) | ('n_other_recent' in repoinfo.keys())): continue
        summary = formatreponame(reponame=f"`{reponame}`", fixedwidth=30)
        if any(repoinfo['recent']):
            recent = summarize(reponame=reponame, commits=repoinfo['recent'])
            summary += recent
        if 'n_other_recent' in repoinfo.keys():
            if repoinfo['n_other_recent'] > 0:
                summary += f"* {repoinfo['n_other_recent']} commits by other users.\n"
        summary += f"* {len(repoinfo['untracked'])} untracked files.\n"
        fullsummary = fullsummary + "\n" + summary
    return fullsummary

In [3]:
# main
base = findrepos(gitdir="~/git")
base = checkrepos(info=base)
today = datetime.now()
aweekago = today - relativedelta(days=+7)
repos = recentcommits(info=base, sdate=aweekago, edate=today, author="bailey")
summary = summarizerecent(repos)

## preview

In [4]:
reponame = 'Chi-MP-data-story'
repo = git.Repo(repos[reponame]['path'])
recent = repos[reponame]['recent']
commit = recent[0]

In [5]:
commit

<git.Commit "ed746774c471ae3c64de57fb29997f286643f7a6">

In [6]:
repo.commit().author.name, repo.commit().author.email

('Bailey', 'bailey@hrdag.org')

In [7]:
commit.message

'pulled one more small section and re-ran notebook\n'

In [8]:
base['PR-Km0']

{'path': '/Users/home/git/PR-Km0',
 'dirty': False,
 'untracked': [],
 'n_other_recent': 2,
 'recent': []}

# meetings +/- 1 week

- Using code from the [TODO-helper `compose`](https://github.com/baileyb0t/TODO-helper/tree/main/compose) task: https://github.com/baileyb0t/TODO-helper/tree/main/compose
- The TODO-helper repo has a `calendar` task that handles downloading the google calendar as an .ics file for use in building this note file. I'm not going to copy or cover that code, just use the .ics file here since I will be incorporating this routine into TODO-helper once polished.

In [9]:
import yaml
from zoneinfo import ZoneInfo
from datetime import datetime
import holidays
from modules.doc import Doc


def read_yaml(fname):
    with open(fname, 'r') as f:
        rules = yaml.safe_load(f)
        f.close()
    return rules


def format_date(from_arg, date=datetime.now()):
    if not from_arg:
        return date.replace(tzinfo=ZoneInfo('US/Pacific'))
    if '-' in date: form = datetime.strptime(date, '%Y-%m-%d')
    else: form = datetime.strptime(date, '%Y%m%d')
    return form.astimezone(ZoneInfo('US/Pacific'))


def prep_out(givendate, outdir):
    if not givendate: today = format_date(from_arg=False)
    else: today = format_date(from_arg=True, date=givendate)
    path = f"{outdir}/{today.strftime('%Y-%m-%d')}"
    today = today
    return path, today


def check_holidays():
    by_county = {holidays.country_holidays(country).get(today)
                 for country in countries}
    by_market = {holidays.financial_holidays(market).get(today)
                 for market in markets}
    found = {v for v in by_county.union(by_market) if v}
    if not any(found): return None
    return found


def add_holidays(notes):
    found = check_holidays()
    label = 'National or financial holiday(s)'
    notes.insert(prefix=formats['notes'], text=f'{label}:\t{found}')
    notes.insert(prefix='', text='')
    return notes

In [10]:
from datetime import date
from icalendar import Calendar
import recurring_ical_events
from modules import doc


def load_cal(icsname):
    with open(icsname, 'rb') as f:
        ecal = Calendar.from_ical(f.read())
    f.close()
    return ecal


def get_event_info(event):
    return {
        'title': str(event['SUMMARY']),
        'timestamp': event.decoded('DTSTART'),
        'date': event.decoded('DTSTART').strftime('%Y-%m-%d'),
        'time': event.decoded('DTSTART').strftime('%H:%M'),
        'weekday': event.decoded('DTSTART').strftime('%a'),
    }


def find_events(ecal, caldate):
    events = []
    for event in recurring_ical_events.of(ecal).at(caldate):
        if (event.decoded('DTSTART').hour < 9) | (event.decoded('DTSTART').hour > 17): continue
        if event.decoded('DTSTART').isoweekday() > 5: continue
        if 'SUMMARY' in event:
            if ' appt' in str(event['SUMMARY']): continue
            info = get_event_info(event)
            if 'LOCATION' in event: info['location'] = str(event['LOCATION'])
            events.append(info)
    return events


def get_events(ecal, caldate):
    events = find_events(ecal, caldate)
    out = []
    for event in events:
        if 'location' in event:
            if 'https' in event['location'].lower(): meet_type='virtual'
            else: meet_type='in person'
        else:
            meet_type='no location set'
        text = f"{event['weekday']} {event['time']}: {event['title']} ({meet_type})"
        out.append(text)
    return sorted(out)


def add_events(notes, events, prefix):
    if not any(events): return notes
    for text in events:
        notes.insert(prefix=prefix,
                     text=text)
    return notes

In [11]:
ecal = load_cal("../../TODO-helper/calendar/output/mpb.ics")
rules = read_yaml("hand/rules.yml")
outdir = "."
notesjson = f"{outdir}/notes.json"

formats = rules['format']
countries = rules['countries'].split()
markets = rules['markets'].split()

path, today = prep_out(givendate=date.today().strftime("%Y%m%d"), outdir="./")
notes = Doc(prefix='# ',
          text=today.strftime('%A, %d %B %Y'),
          path=path,
          dailyday=today.strftime('%Y-%m-%d'))
notes = add_holidays(notes)
notes.insert(prefix=formats['text'], text='')

notes.insert(prefix=formats['header'], text='On my plate')
notes.insert(prefix=formats['subheader'], text='Priorities this week')
notes.insert(prefix=formats['notes'], text='\n')
notes.insert(prefix=formats['subheader'], text='Back-burner this week')
notes.insert(prefix=formats['notes'], text='\n')
notes.insert(prefix=formats['subheader'], text='Back-back-burner')
notes.insert(prefix=formats['notes'], text='\n\n')

notes.to_json(notesjson)

'./notes.json written successfully'

In [12]:
notes = doc.from_json(notesjson)
outdir = "."
notesmd = f"{outdir}/notes.md"

today = datetime.now()
aweekago = today - relativedelta(days=+7)

notes.insert(prefix=formats['header'], text='On the calendar')
notes.insert(prefix=formats['subheader'], text='Last week')
for i in range(0, 7):
    caldate = (aweekago + relativedelta(days=+i)).strftime("%Y%m%d")
    events = get_events(ecal, caldate)
    notes = add_events(notes, events, prefix=formats['meeting_done'])
notes.insert(prefix=formats['text'], text='')

notes.insert(prefix=formats['subheader'], text='This week')
for i in range(0, 7):
    caldate = (today + relativedelta(days=+i)).strftime("%Y%m%d")
    events = get_events(ecal, caldate)
    notes = add_events(notes, events, prefix=formats['meeting'])
notes.insert(prefix=formats['text'], text='\n')

notes.insert(prefix=formats['header'], text='Repo activity')
notes.insert(prefix=formats['text'], text=summary)

notes.to_json(notesjson)
notes.to_md(notesmd)

1

In [13]:
notes

# Sunday, 12 January 2025
- National or financial holiday(s):	None


# On my plate
### Priorities this week
- 

### Back-burner this week
- 

### Back-back-burner
- 


# On the calendar
### Last week
- [x] Mon 09:00: Monday Office Hours (virtual)
- [x] Wed 09:00: TS + BP standing (virtual)
- [x] Thu 14:30: chat w/ MEP (virtual)
- [x] Fri 12:30: testing (in person)

### This week
- [ ] Mon 09:00: Monday Office Hours (virtual)
- [ ] Tue 10:00: ACCESS project sync (virtual)
- [ ] Tue 11:00: team mtg (virtual)
- [ ] Wed 09:00: TS + BP standing (virtual)
- [ ] Wed 11:00: HRDAG, SFPDO, and friends (virtual)
- [ ] Fri 13:00: Code Review (no location set)


# Repo activity

+--------------------------------+
|            `PR-Km0`            |
+--------------------------------+
* 2 commits by other users.
* 0 untracked files.

+--------------------------------+
|      `Chi-MP-data-story`       |
+--------------------------------+
_Committed: Thu 09 Jan_
* [ed746774]: Pulled one more small secti